In [1]:
import pandas as pd
from pandas import DataFrame
import os
import sys
import csv
from pathlib import Path
import seaborn as sns
sns.set()
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
plt.show()
from collections import defaultdict
import os.path as op
import glob
from Bio import SeqIO


os.chdir('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/Genus_consensus_16S_sequence/')
images='/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_images/16S_trees/'
outdir=('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/Genus_consensus_16S_sequence/')
listdir=('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/Genus_consensus_16S_sequence/lists/')
matplotlib.__version__

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/Genus_consensus_16S_sequence/'

### NOT rerun on 2021-12-07 after correcting for salinity of media samples

In [ ]:
Genus=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/Genus_statistics.csv', index_col=0)
Genus=Genus[Genus['#_of_cells'] >=5]
Genus=Genus[Genus['GTDB_genus'] != 'Unclassified']
tenSAG=Genus['GTDB_genus'].tolist()
tenSAG=np.unique(tenSAG)
print(len(tenSAG))

In [ ]:
SAG_data=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/All_GoM_SAGs_1cell_20kb_decon_531normalized_predresp_rate_GTDBclass.csv',
                    index_col=0)
SAG_data

In [ ]:
#create a fasta file for every genus

!rm *.fasta
SSU_dir=glob.glob('/mnt/scgc/simon/microg2p/Data/SAGs_WGS/SSUs/GoM_SAG_SSUs/*.fasta')

for genus in tenSAG:
    df=SAG_data[SAG_data['GTDB_classification'] == genus]
    SAGs=df['name'].to_list()

    with open(genus+'_SSU_sequences.fasta', 'w') as out:
        for i in SAGs:
            for infile in SSU_dir:
                if i in infile:
                    with open(infile, 'r') as infile:
                        out.write(infile.read())
                
         


In [ ]:
# sort the 16S sequences long to short
!for file in *_SSU_sequences.fasta; do seqkit sort $file -o ${file/%SSU_sequences.fasta/sorted.fasta} -l -r; done

# extract just the longest 16S sequence
!for file in *sorted.fasta; do awk '/^>/{if(N)exit;++N;} {print;}' $file > ${file/%sorted.fasta/longest16S.fasta}; done

#remove the sorted files
!rm *sorted.fasta

In [ ]:
# rename the name of the longest sequence so that it is the name of the genus
# This helps with future annotations of a tree where I have to refer to the branch length
fasta_files=glob.glob('*16S.fasta')

for i in fasta_files:
    
    genus=i.split("_lon")[0]
    genus=str(genus)
   # print(genus)
    fasta_in = open(i)

    # Create FASTA output file.
    fasta_out = open(genus+'_16Srep.fasta', 'w')

    for line in fasta_in.readlines():
        if line.startswith('>'):
            contig_id = '>' + str(genus) + '\n'
 #           print(contig_id)
            fasta_out.write(contig_id)
        else:
            fasta_out.write(line)
    fasta_out.close()
    fasta_in.close()


In [ ]:
# combine all the longest representatives into a single fasta file.

!rm Genus_longest_16S_sequence.fasta
!rm *longest16S.fasta
!cat *16Srep.fasta >> Genus_longest_16S_sequence.fasta
!seqkit stats Genus_longest_16S_sequence.fasta